# このノートブックについて

[ノートブックへ](https://www.dropbox.com/s/vsfglb1bkth10i3/20200622-kw-request-handler.ipynb?dl=0)

Bokeh のウィジェットを利用した場合、そのコールバックは原則的に JavaScript で記述する。使い慣れた Python でコッルバックを書きたい、あるいは Python 側にあるデータをアクセスしたい、サーバ側のデータベースにアクセスしたいなどの理由から Python でコールバックを書きたいことも多い。

このノートブックは、JavaScript で書かれた小さなコールバックから、Python の関数を起動し、**あたかも** その関数がウィジェット用のコールバックのように振る舞う仕掛けを作っている。

Python のコールバック関数はウィジェットを直接的に監視することはできない。そこで、この関数が監視できるデータソースを用意し、それを JavaScript のコールバックに受け渡し、コールバックのなかでその内容を更新する。Bokeh は、この JavaScript 側での更新を Python 側の対応するデータソースへの更新として反映する。この Python 側での更新により、Python 側のデータソースを監視している Python で書かれたコールバック関数が駆動される。

In [1]:
import numpy as np
from bokeh.plotting import figure, output_notebook, show
from bokeh.models import HoverTool, ColumnDataSource, CustomJS

以下の JavaScript 用のコッルバックでは、Hover された点の ID を JavaScript のデータ構造に保存された選択されているデータ群 (`cb_data.index.indices`) をPython のデータソースに移植しているらしい。
- (`console.log` で確認したところ)`js_callback` は、Hover とは無関係にマウスの移動のごとに呼ばれるらしい。
- `if` 文はなにを確認しているのか？
- `data_source` が確かに Python のデータソースに該当することを確認すること。


In [2]:
JS_CODE = '''
    if (cb_data.renderer.id==glyph.id) {
      selection.selected.indices = cb_data.index.indices;
    }'''

- `selection` は空のデータソース。ここに Hover ツールで選択されたデータの情報が書き込まれる。

In [3]:
def ask_js_callback_to_call(hover_tool, glyph, py_callback):
    selection = ColumnDataSource()
    selection.selected.on_change('indices', py_callback)

    # 選択された領域が更新されたときに JS コールバックを読む仕組み
    js_callback = CustomJS(args={'selection': selection, 'glyph': glyph}, code=JS_CODE)
    print(hover_tool.callback)
    #hover_tool.callback = js_callback if hover_tool.callback is None else hover_tool.callback + [js_callback]
    hover_tool.callback = js_callback

## Bokeh ドキュメントの構成

- $[0,1]^2$の領域に無作為に青い点を$B$個、赤い点を$R$個ずつ配置する。
- 一見すると赤い点がホバーされたときのコールバック(`py_callback`)が Hover ツールから呼ばれるように見える。しかし、実際に設定されているのは JavaScript のコールバック (`js_callback`) である。`js_callback` が裏に隠れたデータソースの更新 (`indices`イベント) を介して、間接的に `py_callback` が呼ばれるように仕向ける仕掛けになっている。

In [4]:
def Document(doc):
    (B, R) = 5, 5

    fig = figure()
    blue_points = fig.circle(x='x', y='y', fill_color='blue', line_color=None,
                             source=ColumnDataSource(dict(x=np.random.rand(B), y=np.random.rand(B))))
    red_points = fig.circle(x='x', y='y', fill_color='red', line_color=None,
                            source=ColumnDataSource(dict(x=np.random.rand(R), y=np.random.rand(R))))

    def py_callback(action, _, i):
        if len(i) > 0:
            print(f'{action}({i[0]}) -> x={red_points.data_source.data["x"][i][0]:.3f}')

    hover_tool = HoverTool()
    ask_js_callback_to_call(hover_tool, red_points, py_callback)

    fig.add_tools(hover_tool)
    doc.add_root(fig)

In [5]:
output_notebook()
show(Document)

Loading BokehJS ...

None
indices(0) -> x=0.311
indices(3) -> x=0.183
indices(3) -> x=0.183
indices(1) -> x=0.173
indices(2) -> x=0.410
indices(4) -> x=0.495
